In [ ]:
!pip install mysql-connector-python xmltodict pandas

In [ ]:
import xmltodict
import pandas as pd
import mysql.connector
from mysql.connector import Error

DB_CONFIG = {
    'host': 'localhost',  # Detalle de conexion BD
    'user': 'root',
    'password': 'root1234',  # contraseña
    'database': 'PROYECTO'  # Nombre de la base de datos
}

# Función para leer y procesar el archivo XML
def read_and_process_xml(file_path):
    print(f"Leyendo archivo XML: {file_path}...")
    with open(file_path, 'r', encoding='utf-8') as file:
        data = xmltodict.parse(file.read())

    # Imprimir la estructura del XML para depurar
    print(data)

    # Extraer datos relevantes
    users = []
    properties = []

    # Iterar sobre las propiedades en el archivo XML
    for item in data['data']['listing']:
        # Propiedad
        properties.append({
            'estado': item.get('state', 'Desconocido'),  # 'state' en el XML
            'ciudad': item.get('city', 'Desconocida'),  # 'city' en el XML
            'colonia': item.get('colony', 'Desconocida'),  # 'colony' en el XML
            'calle': item.get('street', 'Desconocida'),  # 'street' en el XML
            'numero_exterior': item.get('external_num', None) if item.get('external_num', None) != 'null' else None,
            'tipo_inmueble': item.get('type', 'Desconocido'),  # 'type' en el XML
            'transaccion': item.get('purpose', 'Desconocido'),  # 'purpose' en el XML
            'precio': float(item.get('price', 0.0)),  # 'price' en el XML
            'codigo_proveedor': item.get('code', 'Desconocido'),  # 'code' en el XML
            'correo_contacto': item.get('mail_contact', 'Desconocido'),  # 'mail_contact' en el XML
            'telefono_contacto': item.get('phone_contact', 'Desconocido'),  # 'phone_contact' en el XML
        })

        # Usuario (correo de contacto)
        users.append({'correo_contacto': item.get('mail_contact', 'Desconocido')})

    # Convertir listas a DataFrames
    df_users = pd.DataFrame(users).drop_duplicates()
    df_properties = pd.DataFrame(properties)

    print("Datos procesados:")
    print(f"Usuarios: {len(df_users)}")
    print(f"Propiedades: {len(df_properties)}")

    return df_users, df_properties


In [31]:
# Función para cargar datos en la base de datos MySQL
def load_to_mysql(df_users, df_properties):
    print("Cargando datos en la base de datos MySQL...")
    try:
        connection = mysql.connector.connect(**DB_CONFIG)
        if connection.is_connected():
            cursor = connection.cursor()

            # Insertar usuarios
            for _, row in df_users.iterrows():
                cursor.execute("""
                    INSERT IGNORE INTO usuarios (correo_contacto)
                    VALUES (%s)
                """, (row['correo_contacto'],))

            # Insertar propiedades
            for _, row in df_properties.iterrows():
                cursor.execute("""
                    INSERT INTO propiedades (
                        estado, ciudad, colonia, calle, numero_exterior,
                        tipo_inmueble, transaccion, precio, codigo_proveedor,
                        correo_contacto, telefono_contacto
                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """, (
                    row['estado'], row['ciudad'], row['colonia'], row['calle'],
                    row['numero_exterior'], row['tipo_inmueble'],
                    row['transaccion'], row['precio'], row['codigo_proveedor'],
                    row['correo_contacto'], row['telefono_contacto']
                ))

            # Confirmar los cambios
            connection.commit()
            print("Datos cargados exitosamente en la base de datos.")
    except Error as e:
        print(f"Error al conectar a MySQL: {e}")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

In [ ]:
# Ruta al archivo XML (ajustada para Windows)
file_path = '/Users/jeffersondavid/Documents/proyecto_git/feed.xml'  # Ruta del archivo

# Iniciar el proceso ETL
print("Iniciando proceso ETL...")
df_users, df_properties = read_and_process_xml(file_path)

# Cargar los datos a MySQL
print("Cargando datos a MySQL...")
load_to_mysql(df_users, df_properties)

print("Proceso ETL completado.")